<a href="https://colab.research.google.com/github/eleanarey/ProgramingPractices/blob/main/tfm_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers torch torchvision matplotlib scikit-learn

In [3]:
# Script para Clasificación con ViT extendido con Detección de Fotos Borrosas, Duplicados y Optimización para Dispositivos Móviles

from google.colab import drive
import zipfile
import os
import cv2
from skimage.metrics import structural_similarity as ssim
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTImageProcessor

# Asegurar que todas las dependencias estén definidas
try:
    import cv2
except ImportError:
    raise ImportError("La biblioteca cv2 no está instalada. Usa `pip install opencv-python` para instalarla.")

try:
    from skimage.metrics import structural_similarity as ssim
except ImportError:
    raise ImportError("La biblioteca scikit-image no está instalada. Usa `pip install scikit-image` para instalarla.")


In [4]:
from google.colab import drive
import zipfile
import os

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Ruta al archivo ZIP en Google Drive
dataset_zip_path = "/content/drive/MyDrive/Colab_Notebooks/dataset/images/tiny-imagenet-200.zip"
extract_path = "/content/dataset/tiny-imagenet-200"

# Verificar si el archivo ZIP existe
if not os.path.exists(dataset_zip_path):
    raise FileNotFoundError(f"El archivo {dataset_zip_path} no existe. Verifica la ruta.")

# Crear la carpeta de extracción y descomprimir
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extraído en: {extract_path}")


Mounted at /content/drive
Dataset extraído en: /content/dataset/tiny-imagenet-200


In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Redimensionar
    transforms.ToTensor(),           # Convertir a tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalización
])

# Rutas al dataset descomprimido
train_dir = os.path.join(extract_path, "tiny-imagenet-200/train")
val_dir = os.path.join(extract_path, "tiny-imagenet-200/val")

# Crear datasets y DataLoaders
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Clases: {train_dataset.classes}")


Clases: ['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815834', 'n02823428', 'n02837789', 'n02841315', 'n0

In [6]:
from transformers import ViTForImageClassification, ViTImageProcessor
import torch

# Configuración del modelo
num_classes = len(train_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelo ViT preentrenado
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=num_classes,  # Configurar para 200 clases
    ignore_mismatched_sizes=True  # Ignorar discrepancias de tamaño
)

model = model.to(device)

# Optimización
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([200]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([200, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Entrenamiento
for epoch in range(1):  # Ajusta el número de épocas según sea necesario
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Época {epoch + 1}, Pérdida: {running_loss / len(train_loader)}")


Época 1, Pérdida: 0.9228811931490898
Época 2, Pérdida: 0.24542514461368323
Época 3, Pérdida: 0.1313110634934157
Época 4, Pérdida: 0.0858390294598043
Época 5, Pérdida: 0.06696780223987997


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab_Notebooks/vit_tiny_imagenet_200.pth")


In [ ]:
# Detección de Fotos Borrosas
def is_blurry(image_path, threshold=100):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"La imagen {image_path} no existe.")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"No se pudo leer la imagen {image_path}.")
    laplacian_var = cv2.Laplacian(image, cv2.CV_64F).var()
    return laplacian_var < threshold

image_path = "/content/dataset/tiny-imagenet-200/train/"
try:
    print(f"¿La imagen está borrosa? {'Sí' if is_blurry(image_path) else 'No'}")
except Exception as e:
    print(f"Error en la detección de borrosidad: {e}")



Error en la detección de borrosidad: La imagen /content/dataset/tiny-imagenet-200/train/ no existe.


In [ ]:
# Detección de Duplicados
def are_images_similar(image1_path, image2_path, threshold=0.9):
    if not os.path.exists(image1_path) or not os.path.exists(image2_path):
        raise FileNotFoundError("Una o ambas imágenes no existen.")
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    if img1 is None or img2 is None:
        raise ValueError("No se pudo leer una o ambas imágenes.")
    similarity, _ = ssim(img1, img2, full=True)
    return similarity > threshold

image1 = "/content/dataset/tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG"
image2 = "/content/dataset/tiny-imagenet-200/train/n01443537/images/n01443537_1.JPEG"

try:
    print(f"¿Las imágenes son duplicadas? {'Sí' if are_images_similar(image1, image2) else 'No'}")
except Exception as e:
    print(f"Error en la detección de duplicados: {e}")

Error en la detección de duplicados: Una o ambas imágenes no existen.


In [ ]:
def optimize_model_for_mobile(model, output_path):
    # Configurar el modelo para evaluación
    model.eval()

    # Configurar la cuantización con esquema compatible
    model.qconfig = torch.quantization.get_default_qconfig('qnnpack')  # Cambiar a qnnpack
    print("Preparando el modelo para la cuantización...")

    # Preparar y convertir el modelo
    model_prepared = torch.quantization.prepare(model)
    model_int8 = torch.quantization.convert(model_prepared)

    # Guardar el modelo optimizado
    scripted_model = torch.jit.script(model_int8)
    scripted_model.save(output_path)
    print(f"Modelo optimizado guardado en: {output_path}")

# Ruta para guardar el modelo optimizado
optimized_model_path = "/content/drive/MyDrive/Colab_Notebooks/vit_tiny_imagenet_200_optimized.pt"
optimize_model_for_mobile(model, optimized_model_path)


Preparando el modelo para la cuantización...


RuntimeError: Can't redefine method: forward on class: __torch__.transformers.models.vit.modeling_vit.ViTEmbeddings (of Python compilation unit at: 0x5a7135991ee0)

In [ ]:
pip install torch torchvision


In [ ]:
from torchvision.models import resnet18

# Cargar un modelo ResNet preentrenado
resnet_model = resnet18(pretrained=True)
resnet_model.eval()

# Cuantización
resnet_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
resnet_prepared = torch.quantization.prepare(resnet_model)
resnet_int8 = torch.quantization.convert(resnet_prepared)

# Guardar modelo cuantizado
quantized_model_path = "/content/drive/MyDrive/Colab_Notebooks/resnet18_quantized.pt"
torch.jit.save(torch.jit.script(resnet_int8), quantized_model_path)
print(f"Modelo cuantizado guardado en: {quantized_model_path}")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 85.6MB/s]


Modelo cuantizado guardado en: /content/drive/MyDrive/Colab_Notebooks/resnet18_quantized.pt
